In [8]:
%load_ext autoreload
%autoreload
%matplotlib inline

import pandas as pd
import numpy as np
import _pickle as pickle
from IPython.core.debugger import set_trace
from tqdm import tqdm_notebook
import json
import math
import os
import re
import datetime
from time import sleep
from sklearn import preprocessing
from tqdm.notebook import tqdm

import gensim
from gensim.models import Word2Vec, Doc2Vec, FastText
from gensim.corpora import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.matutils import sparse2full
from gensim.models.doc2vec import TaggedDocument

import pytz
KST = pytz.timezone('Asia/Seoul')

import snscrape
import snscrape.modules.twitter as sntwitter
import gspread
from oauth2client.service_account import ServiceAccountCredentials

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 구글시트에서 트위터 검색쿼리 데이터 가져오기

In [2]:
scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive',
]

json_file_name = 'sideb-proejct-066e081b7bcd.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1roV5_RPFtJ6z40I6EYGRmqx01Fe6JOOmaJ6nbDrfmkw/edit#gid=2020655845'

# 스프레스시트 문서 가져오기 
doc = gc.open_by_url(spreadsheet_url)

# 시트 선택하기
worksheet = doc.worksheet('20210913')
values = worksheet.get_all_values()
header, rows = values[0], values[1:]

df = pd.DataFrame(rows, columns=header)
df = df[['구분','한글명','영문명 full','영문명 short','트위터 검색쿼리','검색잘됨']].set_index('영문명 short')
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [175]:
user_download_to = 'scrapped/twitter/user'
feed_download_to = 'scrapped/twitter/feed'
n_max = 10
since = '2021-03-01'
until = '2021-08-31'
baseqry = '032c OR 032씨'
# qry = '{baseqry} until:{until} lang:ko'.format(baseqry=baseqry, until=until)
qry = '{baseqry} since:{since} until:{until} lang:ko'.format(baseqry=baseqry, since=since, until=until); qry

'032c OR 032씨 since:2021-03-01 until:2021-08-31 lang:ko'

In [9]:
def convert2serialize(obj):
    if isinstance(obj, dict):
        return { k: convert2serialize(v) for k, v in obj.items() }
    
    elif hasattr(obj, "_ast"):
        return convert2serialize(obj._ast())
    
    elif not isinstance(obj, str) and hasattr(obj, "__iter__"):
        return [ convert2serialize(v) for v in obj ]
    
    elif hasattr(obj, "__dict__"):
        return {
            k: convert2serialize(v)
            for k, v in obj.__dict__.items()
            if not callable(v) and not k.startswith('_')
        }
    
    elif isinstance(obj, datetime.date):
        return obj.astimezone(KST).isoformat()
    
    else:
        return obj

In [88]:
tweets_list = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i, _tw in enumerate(sntwitter.TwitterSearchScraper(qry).get_items()):
    if i > n_max:
        break
        
    tweets_list.append(_tw)
#     tweets_list1.append([tweet.date, tweet.id, tweet.content, tweet.user.username])

In [83]:
tweets_list

[Tweet(url='https://twitter.com/www87mmcokr/status/1432176249565286401', date=datetime.datetime(2021, 8, 30, 3, 0, 1, tzinfo=datetime.timezone.utc), content='Mmlg l 4TH DELIVERY\n\n&lt;WEFLAG&gt; 추가 컬렉션이 오픈되었습니다.🚩\n레트로 스포티즘과 뉴프레피를 해석한 엠엠엘지의 컬렉션은\n지금 팔칠엠엠 공식 온/오프라인 스토어에서 만나보실 수 있어요!\n\n🌏 https://t.co/ZBssYq7iiL\n🏠 마포구 어울마당로 94-8\n\n#87MM #Mmlg https://t.co/GpHzdfpYNx', renderedContent='Mmlg l 4TH DELIVERY\n\n&lt;WEFLAG&gt; 추가 컬렉션이 오픈되었습니다.🚩\n레트로 스포티즘과 뉴프레피를 해석한 엠엠엘지의 컬렉션은\n지금 팔칠엠엠 공식 온/오프라인 스토어에서 만나보실 수 있어요!\n\n🌏 87mm.co.kr\n🏠 마포구 어울마당로 94-8\n\n#87MM #Mmlg https://t.co/GpHzdfpYNx', id=1432176249565286401, user=User(username='www87mmcokr', id=380846167, displayname='87MM_OFFICIAL', description='Newwave Street Culture.', rawDescription='Newwave Street Culture.', descriptionUrls=None, verified=False, created=datetime.datetime(2011, 9, 27, 10, 8, 20, tzinfo=datetime.timezone.utc), followersCount=9104, friendsCount=0, statusesCount=527, favouritesCount=154, listedCount=37, mediaCount=211, lo

In [114]:
idx = 0
tw = convert2serialize(tweets_list[idx])
tw['date'] = tw['date'].astimezone(KST).isoformat()
tw['user']['created'] = tw['user']['created'].astimezone(KST).isoformat();tw

{'url': 'https://twitter.com/www87mmcokr/status/1432176249565286401',
 'date': '2021-08-30T12:00:01+09:00',
 'content': 'Mmlg l 4TH DELIVERY\n\n&lt;WEFLAG&gt; 추가 컬렉션이 오픈되었습니다.🚩\n레트로 스포티즘과 뉴프레피를 해석한 엠엠엘지의 컬렉션은\n지금 팔칠엠엠 공식 온/오프라인 스토어에서 만나보실 수 있어요!\n\n🌏 https://t.co/ZBssYq7iiL\n🏠 마포구 어울마당로 94-8\n\n#87MM #Mmlg https://t.co/GpHzdfpYNx',
 'renderedContent': 'Mmlg l 4TH DELIVERY\n\n&lt;WEFLAG&gt; 추가 컬렉션이 오픈되었습니다.🚩\n레트로 스포티즘과 뉴프레피를 해석한 엠엠엘지의 컬렉션은\n지금 팔칠엠엠 공식 온/오프라인 스토어에서 만나보실 수 있어요!\n\n🌏 87mm.co.kr\n🏠 마포구 어울마당로 94-8\n\n#87MM #Mmlg https://t.co/GpHzdfpYNx',
 'id': 1432176249565286401,
 'user': {'username': 'www87mmcokr',
  'id': 380846167,
  'displayname': '87MM_OFFICIAL',
  'description': 'Newwave Street Culture.',
  'rawDescription': 'Newwave Street Culture.',
  'descriptionUrls': None,
  'verified': False,
  'created': '2011-09-27T19:08:20+09:00',
  'followersCount': 9104,
  'friendsCount': 0,
  'statusesCount': 527,
  'favouritesCount': 154,
  'listedCount': 37,
  'mediaCount': 211,
  'loca

In [115]:
user = tw['user']; user

{'username': 'www87mmcokr',
 'id': 380846167,
 'displayname': '87MM_OFFICIAL',
 'description': 'Newwave Street Culture.',
 'rawDescription': 'Newwave Street Culture.',
 'descriptionUrls': None,
 'verified': False,
 'created': '2011-09-27T19:08:20+09:00',
 'followersCount': 9104,
 'friendsCount': 0,
 'statusesCount': 527,
 'favouritesCount': 154,
 'listedCount': 37,
 'mediaCount': 211,
 'location': '대한민국 서울',
 'protected': False,
 'linkUrl': 'http://87mm.co.kr',
 'linkTcourl': 'https://t.co/KjpxakTGkk',
 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1361594043008880642/NUNxbk63_normal.jpg',
 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/380846167/1628652099'}

In [116]:
tw['user'] = user['username']; tw

{'url': 'https://twitter.com/www87mmcokr/status/1432176249565286401',
 'date': '2021-08-30T12:00:01+09:00',
 'content': 'Mmlg l 4TH DELIVERY\n\n&lt;WEFLAG&gt; 추가 컬렉션이 오픈되었습니다.🚩\n레트로 스포티즘과 뉴프레피를 해석한 엠엠엘지의 컬렉션은\n지금 팔칠엠엠 공식 온/오프라인 스토어에서 만나보실 수 있어요!\n\n🌏 https://t.co/ZBssYq7iiL\n🏠 마포구 어울마당로 94-8\n\n#87MM #Mmlg https://t.co/GpHzdfpYNx',
 'renderedContent': 'Mmlg l 4TH DELIVERY\n\n&lt;WEFLAG&gt; 추가 컬렉션이 오픈되었습니다.🚩\n레트로 스포티즘과 뉴프레피를 해석한 엠엠엘지의 컬렉션은\n지금 팔칠엠엠 공식 온/오프라인 스토어에서 만나보실 수 있어요!\n\n🌏 87mm.co.kr\n🏠 마포구 어울마당로 94-8\n\n#87MM #Mmlg https://t.co/GpHzdfpYNx',
 'id': 1432176249565286401,
 'user': 'www87mmcokr',
 'replyCount': 0,
 'retweetCount': 3,
 'likeCount': 3,
 'quoteCount': 0,
 'conversationId': 1432176249565286401,
 'lang': 'ko',
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'sourceUrl': 'https://mobile.twitter.com',
 'sourceLabel': 'Twitter Web App',
 'outlinks': ['http://87mm.co.kr'],
 'tcooutlinks': ['https://t.co/ZBssYq7iiL'],
 'media': [{'previe

In [121]:
feed_fname = feed_download_to + '/' + tw['date'].replace(':',';') + '.json'
user_fname = user_download_to + '/' + user['username'] + '.json'
print(feed_fname)
print(user_fname)

scrapped/twitter/feed/2021-08-30T12;00;01+09;00.json
scrapped/twitter/user/www87mmcokr.json


In [125]:
# 같은 피드가 없으면 파일로 저장해라
if not os.path.isfile(feed_fname):
    with open(feed_fname, 'w', encoding='UTF-8-sig') as f:
        json.dump(tw, f, ensure_ascii=False)

In [126]:
# 같은 유저가 있던말던 무조건 새 파일로 저장해라 (팔로워수 등은 업데이트 되므로)
with open(user_fname, 'w', encoding='UTF-8-sig') as f:
    json.dump(user, f, ensure_ascii=False)

In [6]:
user_download_to = 'scrapped/twitter/user'
feed_download_to = 'scrapped/twitter/feed'
# n_max = 10
since = '2021-03-01'
until = '2021-08-31'
# baseqry = '032c OR 032씨'
# qry = '{baseqry} until:{until} lang:ko'.format(baseqry=baseqry, until=until)
qry = '{baseqry} since:{since} until:{until} lang:ko'.format(baseqry=baseqry, since=since, until=until); qry

'032c OR 032씨 since:2021-03-01 until:2021-08-31 lang:ko'

In [17]:
def record(tw, brand):
    _tw = convert2serialize(tw)
    _user = _tw['user']
    _tw['user'] = _user['username']
    _tw['brands'] = []
    
    feed_fname = feed_download_to + '/' + _tw['date'].replace(':', ';') + '.json'
    user_fname = user_download_to + '/' + _user['username'] + '.json'

    # 같은 피드가 없으면 파일로 저장해라
    if not os.path.isfile(feed_fname):
        with open(feed_fname, 'w', encoding='UTF-8-sig') as f:
            _tw['brands'].append(brand)
            json.dump(_tw, f, ensure_ascii=False)
            
    # 같은 피드가 있으면, 해당 브랜드를 추가로 입력해라
    else:
        with open(feed_fname, 'r', encoding='UTF-8-sig') as f:
            _tw = json.load(f)
            
            if brand not in _tw['brands']:
                _tw['brands'].append(brand)
            
        with open(feed_fname, 'w', encoding='UTF-8-sig') as f:            
            json.dump(_tw, f, ensure_ascii=False)        
            
    # 같은 유저가 있던말던 무조건 새 파일로 저장해라 (팔로워수 등은 업데이트 되므로)
    with open(user_fname, 'w', encoding='UTF-8-sig') as f:
        json.dump(_user, f, ensure_ascii=False)        

In [10]:
qrys = df[df['검색잘됨']=='1']['트위터 검색쿼리']; qrys

영문명 short
032c                                            032c OR 032씨
87mm                            87엠엠 OR 87앰앰 OR 팔칠앰앰 OR 팔칠엠엠
99percentis             99percentis OR 99퍼센트이즈 OR 나인티나인퍼센트이즈
jwanderson                   jwanderson OR jw앤더슨 OR 제이더블유앤더슨
lostmanagementcities                                     엘엠씨
                                        ...                 
tictok                                          tictok OR 틱톡
playmobil                                 playmobil OR 플레이모빌
playstation                     playstation OR 플레이스테이션 OR 플스
hansatoys                              "hansatoys" OR "한사토이"
hottoys                                      hottoys OR 핫토이즈
Name: 트위터 검색쿼리, Length: 569, dtype: object

In [20]:
for brand, baseqry in tqdm(list(qrys.items())[:]):
    qry = '{baseqry} since:{since} until:{until} lang:ko'.format(baseqry=baseqry, since=since, until=until)
    #print(qry)
    
    for i, _tw in enumerate(sntwitter.TwitterSearchScraper(qry).get_items()):
        record(_tw, brand)
    
    sleep(10)
    #sleep(0.001)

  0%|          | 0/569 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [16]:
1 not in [1,2,3]

False